Predicting Spam with Capital Letter Frequency

REMEMBER: upload both an html and .ipynb file to the assignment

Question : Can the crediblility of mail be catagorized based on the use of capital letters and keywords in the message?

Introduction:

Spam mail, or unsolicited messages sent en mass to random recipients have grown in volume in recent years following the convience and anonymity of the internet [Kaddoura et al., 2022]. Ranging from harmless buisness advertisements to scams taking advantage of vulnerable groups, this phenonmenon imposes varying degrees of economic and emotional harm on a vast number of e-mail users [Kaddoura et al., 2022]. Attention grabbing word and an overuse of capital letters are commen features of spam mail [Mujtaba et al., 2017] and mail-filtering programs have long been proposed as a potential solution to reduce user interaction with spam mail [Kaddoura et al., 2022]. Since many of these programS use lexical and term-based features in identifying spam mail [Mujtaba et al., 2017], the reliability of these identifiers are important to the accuracy of spam mail detectors. We will be investigating if the crediblility of mail be catagorized based on the use of capital letters and keywords in email using the UCI Machine Learing Repository's Spambase dataset which is composed of emails filled as spam by recipients as well as personal and work related non-spam emails. The dataset provides infomation regarding the longest run length of capital letters throughout the message (in the capital_run_length_longest column) as well percentage of certain characters in the message such as "free" and "!". We will be analyzing the accuracy of catagorizing spam mail base on the percentage of the strings "000", "free" and "credit" as well as the longest run of capital letters in the message. Our exploratory data ananysis showed that these freatures had a significan mean ration between spam and non spam emails and are likely good identifiers.




In [1]:
import pandas as pd
from collections import ChainMap
from sklearn.model_selection import train_test_split

In [2]:
spam = pd.read_csv("spambase.data", header = None)
spam

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.000,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.010,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4596,0.31,0.00,0.62,0.0,0.00,0.31,0.00,0.00,0.00,0.00,...,0.000,0.232,0.0,0.000,0.000,0.000,1.142,3,88,0
4597,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.353,0.000,0.000,1.555,4,14,0
4598,0.30,0.00,0.30,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.102,0.718,0.0,0.000,0.000,0.000,1.404,6,118,0
4599,0.96,0.00,0.00,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.057,0.0,0.000,0.000,0.000,1.147,5,78,0


In [3]:
spam_titles = pd.read_csv("spambase.names", skiprows = 31)
spam_titles
spam_titles_split = spam_titles["1"].str.split(":", expand = True)
spam_titles_split
spam_headers = spam_titles_split[[0]].to_dict()
spam_headers = dict(ChainMap(*spam_headers.values()))
spam_headers
spam = spam.rename(columns = spam_headers).rename(columns = {57: "type"})

spam

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,type
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.000,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.010,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4596,0.31,0.00,0.62,0.0,0.00,0.31,0.00,0.00,0.00,0.00,...,0.000,0.232,0.0,0.000,0.000,0.000,1.142,3,88,0
4597,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.353,0.000,0.000,1.555,4,14,0
4598,0.30,0.00,0.30,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.102,0.718,0.0,0.000,0.000,0.000,1.404,6,118,0
4599,0.96,0.00,0.00,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.057,0.0,0.000,0.000,0.000,1.147,5,78,0


In [10]:
spam_train, spam_test = train_test_split(spam, train_size = 0.75, stratify = spam["type"])
spam_test

word_freq_make                  0.113319
word_freq_address               0.197107
word_freq_all                   0.294787
word_freq_3d                    0.042832
word_freq_our                   0.316247
word_freq_over                  0.084344
word_freq_remove                0.107819
word_freq_internet              0.080287
word_freq_order                 0.075977
word_freq_mail                  0.259774
word_freq_receive               0.051251
word_freq_will                  0.559861
word_freq_people                0.090104
word_freq_report                0.065004
word_freq_addresses             0.042632
word_freq_free                  0.254057
word_freq_business              0.132911
word_freq_email                 0.186056
word_freq_you                   1.731590
word_freq_credit                0.096681
word_freq_your                  0.780217
word_freq_font                  0.118367
word_freq_000                   0.099209
word_freq_money                 0.094023
word_freq_hp    

In [25]:
spam_mail = spam_train[spam_train["type"] == 1]
non_spam = spam_train[spam_train["type"] == 0]
mean_spam = pd.DataFrame(spam_mail.mean().reset_index()).rename(columns = {"index": "variable", 0: "mean_value_spam"})
mean_non_spam = pd.DataFrame(non_spam.mean().reset_index()).rename(columns = {"index": "variable", 0: "mean_value_normal"})
mean_non_spam
compair = mean_spam.merge(mean_non_spam, on = "variable")
compair

,variable,mean_value_spam,mean_value_normal
0,word_freq_make,0.145769,0.072941
1,word_freq_address,0.158580,0.257150
2,word_freq_all,0.395504,0.198235
3,word_freq_3d,0.184202,0.000665
4,word_freq_our,0.506946,0.183453
5,word_freq_over,0.185614,0.043955
6,word_freq_remove,0.278675,0.010832
7,word_freq_internet,0.219823,0.044625
8,word_freq_order,0.178043,0.040646
9,word_freq_mail,0.350861,0.155772


In [ ]:
capitals_comparison = 

Works Cited:

    Kaddoura, S., Chandrasekaran, G., Elena Popescu, D., & Duraisamy, J. H. (2022). A systematic literature review on spam content detection and classification. PeerJ Computer Science, 8, e830. https://doi.org/10.7717/peerj-cs.830

    Mujtaba, G., Shuib, L., Raj, R. G., Majeed, N., & Al-Garadi, M. A. (2017). Email Classification Research Trends: Review and Open Issues. IEEE Access, 5, 9044–9064. https://doi.org/10.1109/access.2017.2702187